## Semantic Kernel: Ramp-Up based on SK's Documentation

To get the latest version of SK Python package, use:

``` bash
pip install --upgrade semantic-kernel
```

## 📒 Notebook 6: Plugins

### 🪜 Step 1: Configure environment

In [1]:
# Import required packages
import os
import logging
from typing import Dict, Any
from urllib.parse import urlparse, parse_qs

from semantic_kernel import Kernel
from semantic_kernel.contents import ChatHistory
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions import KernelArguments
from semantic_kernel.functions.kernel_function_from_method import KernelFunctionFromMethod

# For Azure Logic Apps management
from azure.identity import DefaultAzureCredential
from azure.mgmt.web import WebSiteManagementClient

In [2]:
# Set Azure OpenAI backend variables
AOAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_API_DEPLOY")
AOAI_ENDPOINT = os.getenv("AZURE_OPENAI_API_BASE")
AOAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

# Set Azure Logic Apps variables
SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID")
RESOURCE_GROUP = os.getenv("RESOURCE_GROUP_NAME", "App_LogicApp")
LOGIC_APP_NAME = "Laziz-StandardLA"     # Your Standard Logic App name
WORKFLOW_NAME = "lazizdemologicapp"     # Your workflow inside the Standard Logic App
TRIGGER_NAME = "HTTP-trigger-standard"  # Your HTTP trigger name

### 🪜 Step 2: Generate OpenAPI Spec for Logic App

In [3]:
# Define helper class for Standard Logic Apps OpenAPI spec generation
class StandardLogicAppsOpenAPIHelper:
    """
    Helper class to generate OpenAPI spec for Standard Logic Apps.
    """
    
    def __init__(self, subscription_id: str, resource_group: str, credential=None):
        if credential is None:
            credential = DefaultAzureCredential()
        
        self.subscription_id = subscription_id
        self.resource_group = resource_group
        self.credential = credential
        self.web_client = WebSiteManagementClient(credential, subscription_id)
    
    def get_callback_url(self, logic_app_name: str, workflow_name: str, trigger_name: str) -> str:
        """Get the callback URL with SAS token for a Standard Logic App workflow trigger."""
        try:
            callback = self.web_client.workflow_triggers.list_callback_url(
                resource_group_name = self.resource_group,
                name = logic_app_name,
                workflow_name = workflow_name,
                trigger_name = trigger_name
            )
            if callback.value is None:
                raise ValueError(f"No callback URL returned for workflow '{workflow_name}' in Logic App '{logic_app_name}'.")
            print(f"✓ Retrieved callback URL for workflow: {workflow_name}")
            return callback.value
        except Exception as e:
            print(f"✗ Error getting callback URL: {e}")
            raise
    
    def create_openapi_spec(self, callback_url: str, operation_description: str = None) -> str:
        """
        Create OpenAPI 3.0 spec with explicit SAS parameters for direct invocation.
        """
        if operation_description is None:
            operation_description = "Get weather forecast for a specified location"
        from urllib.parse import urlparse, parse_qs
        parsed_url = urlparse(callback_url)
        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
        query_params = parse_qs(parsed_url.query)
        parameters_yaml = []
        for param_name, param_values in query_params.items():
            if param_values:
                param_value = param_values[0]
                param_entry = f"""        - name: {param_name}
          in: query
          required: true
          schema:
            type: string
            default: '{param_value}'"""
                parameters_yaml.append(param_entry)
        parameters_section = "\n".join(parameters_yaml)
        openapi_yaml = f"""openapi: 3.0.0
info:
  title: Standard Logic App Weather API
  version: 1.0.0
  description: Weather forecast service via Azure Standard Logic App
servers:
  - url: {base_url}
paths:
  /:
    post:
      operationId: getWeatherForecast
      summary: Get weather forecast
      description: {operation_description}
      parameters:
{parameters_section}
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                Location:
                  type: string
                  description: The location to get weather forecast for
              required:
                - Location
      responses:
        '200':
          description: Successful response
"""
        print("✓ OpenAPI spec with auth parameters created successfully")
        return openapi_yaml

print("✓ StandardLogicAppsOpenAPIHelper class defined")

✓ StandardLogicAppsOpenAPIHelper class defined


In [4]:
# Initialise the helper
logic_helper = StandardLogicAppsOpenAPIHelper(
    subscription_id = SUBSCRIPTION_ID,
    resource_group = RESOURCE_GROUP
)

# Get callback URL
callback_url = logic_helper.get_callback_url(
    logic_app_name = LOGIC_APP_NAME,
    workflow_name = WORKFLOW_NAME,
    trigger_name = TRIGGER_NAME
)

# Generate OpenAPI spec
openapi_spec = logic_helper.create_openapi_spec(
    callback_url = callback_url,
    operation_description = "Get current weather forecast for any location worldwide"
)

print("\n" + "="*70)
print("OpenAPI Specification Preview:")
print("="*70)
print(openapi_spec[:70] + "...")

✓ Retrieved callback URL for workflow: lazizdemologicapp
✓ OpenAPI spec with auth parameters created successfully

OpenAPI Specification Preview:
openapi: 3.0.0
info:
  title: Standard Logic App Weather API
  version...


### 🪜 Step 3: Add Logic App's OpenAPI plug-in

In [5]:
# Initialise kernel
kernel = Kernel()

In [6]:
# Configure logging
logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

In [7]:
# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    deployment_name = AOAI_DEPLOYMENT,
    endpoint = AOAI_ENDPOINT,
    api_version = AOAI_API_VERSION,
    service_id = "azure_openai_chat",
)

kernel.add_service(chat_completion)

In [8]:
# Add the OpenAPI plugin to the kernel.
try:
    import os
    spec_file_path = os.path.join(os.getcwd(), "standard_logic_app_openapi.yaml")
    with open(spec_file_path, 'w') as f:
        f.write(openapi_spec)
    print(f"✓ OpenAPI spec saved locally")

    plugin = kernel.add_plugin_from_openapi(
        plugin_name="WeatherPlugin",
        openapi_document_path=spec_file_path
    )
    print("✓ OpenAPI plugin 'WeatherPlugin' added to kernel successfully")
    print(f"✓ Available functions: {list(plugin.functions.keys())}")
except Exception as e:
    print(f"✗ Error during plugin setup: {e}")
    import traceback
    traceback.print_exc()

✓ OpenAPI spec saved locally
✓ OpenAPI plugin 'WeatherPlugin' added to kernel successfully
✓ Available functions: ['getWeatherForecast']


### 🪜 Step 4: Invoke a prompt and observe telemetry in App Insights

In [9]:
# Test the plugin by calling the function directly
try:
    # Get the function from the plugin
    weather_function = kernel.plugins["WeatherPlugin"]["getWeatherForecast"]
    
    # Parse the SAS tokens from the callback URL and normalize their names
    from urllib.parse import urlparse, parse_qs
    parsed_url = urlparse(callback_url)
    query_params = parse_qs(parsed_url.query)

    # Create arguments, replacing hyphens with underscores in the SAS token names
    arguments = KernelArguments(
        Location="London",
        **{k.replace("-", "_"): v[0] for k, v in query_params.items()}
    )
    
    # Invoke the function with all required arguments
    result = await weather_function.invoke(kernel, arguments)
    
    print("="*70)
    print("Direct Function Call Result:")
    print("="*70)
    print(f"Location: London")
    print(f"Response: {result}")
    
except Exception as e:
    print(f"✗ Error invoking function: {e}")
    import traceback
    traceback.print_exc()

Direct Function Call Result:
Location: London
Response: {"responses":{"daily":{"day":{"cap":"Rain showers","pvdrCap":"Rain showers","pvdrWindDir":"241","pvdrWindSpd":"15","icon":23,"symbol":"d2200","pvdrIcon":"23","urlIcon":"http://img-s-msn-com.akamaized.net/tenant/amp/entityid/AAehOqC.img","precip":80.0,"wx":"-RA","sky":"SCT","windDir":241,"windSpd":15.0,"summary":"Watch for scattered rain showers.  The high will be 13°.","summaries":["Watch for scattered rain showers."," The high will be 13°."]},"night":{"cap":"Light rain showers","pvdrCap":"Light rain showers","pvdrWindDir":"218","pvdrWindSpd":"16","icon":46,"symbol":"n2100","pvdrIcon":"46","urlIcon":"http://img-s-msn-com.akamaized.net/tenant/amp/entityid/AAehyQD.img","precip":59.0,"wx":"-RA","sky":"SCT","windDir":218,"windSpd":16.0,"summary":"There will be scattered light rain showers.  The low will be 8°.","summaries":["There will be scattered light rain showers."," The low will be 8°."]},"pvdrCap":"Rain showers","pvdrWindDir":"2

In [10]:
# Enable auto function calling
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings

# Create execution settings with auto function calling
execution_settings = OpenAIChatPromptExecutionSettings(
    service_id="azure_openai_chat",
    max_tokens=2000,
    temperature=0.7,
    top_p=0.8,
)

# Create a chat history
chat_history = ChatHistory()
chat_history.add_system_message(
    "You are a helpful weather assistant. When asked about weather, "
    "use the getWeatherForecast function to get current weather information."
)
chat_history.add_user_message("What's the weather like in Paris?")

print("Testing with auto function calling...")
print("="*70)

Testing with auto function calling...


In [11]:
# Get chat completion with auto function calling
try:
    # Get the chat completion service
    chat_service = kernel.get_service("azure_openai_chat")
    
    # Get response (SK will automatically call the function if needed)
    response = await chat_service.get_chat_message_content(
        chat_history=chat_history,
        settings=execution_settings,
        kernel=kernel
    )
    
    print("Chat Response:")
    print("="*70)
    print(f"User: What's the weather like in Paris?")
    print(f"\nAssistant: {response}")
    
    # Add response to history for continued conversation
    chat_history.add_assistant_message(str(response))
    
except Exception as e:
    print(f"✗ Error during chat completion: {e}")
    import traceback
    traceback.print_exc()

Chat Response:
User: What's the weather like in Paris?

Assistant: Could you please specify if you want the current weather or the forecast for a specific date?


In [12]:
# Ask a follow-up question
try:
    chat_history.add_user_message("How about Tokyo?")
    
    response = await chat_service.get_chat_message_content(
        chat_history=chat_history,
        settings=execution_settings,
        kernel=kernel
    )
    
    print("\nFollow-up Response:")
    print("="*70)
    print(f"User: How about Tokyo?")
    print(f"\nAssistant: {response}")
    
except Exception as e:
    print(f"✗ Error during follow-up: {e}")


Follow-up Response:
User: How about Tokyo?

Assistant: Could you please specify if you want the current weather or the forecast for a specific date in Tokyo?


In [13]:
# Display the full conversation
print("\nFull Conversation History:")
print("="*70)

for message in chat_history.messages:
    role = message.role.value.capitalize()
    content = message.content
    print(f"\n{role}: {content}")
    print("-"*70)


Full Conversation History:

System: You are a helpful weather assistant. When asked about weather, use the getWeatherForecast function to get current weather information.
----------------------------------------------------------------------

User: What's the weather like in Paris?
----------------------------------------------------------------------

Assistant: Could you please specify if you want the current weather or the forecast for a specific date?
----------------------------------------------------------------------

User: How about Tokyo?
----------------------------------------------------------------------
